# 🔧 04: Tool Calling Basics

Learn how to extend your LLM's capabilities by giving it access to built-in tools for calculations, text transformations, and more.

## 📋 Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Understand what tools are and why they're useful
- [ ] Use the built-in `math_calculator` tool for arithmetic
- [ ] Use the `text_transformer` tool for text manipulation
- [ ] Use the `char_counter` tool for text analysis
- [ ] See how tools are automatically executed by the SDK
- [ ] Inspect tool calls in full responses
- [ ] Combine multiple tools in a single conversation

## 🎯 Prerequisites

- Completed notebooks 02 (Basic Chat) and 03 (Conversation History)
- Understanding of chat and conversation history
- LM Studio running with a model that supports function calling

## ⏱️ Estimated Time: 15 minutes

## 1️⃣ What Are Tools?

**Tools** (also called function calling) let LLMs interact with external functions. This solves key limitations:

| Without Tools | With Tools |
|---------------|------------|
| ❌ Bad at precise math | ✅ Use calculator for exact answers |
| ❌ Can't access real-time data | ✅ Call APIs for current information |
| ❌ Can't modify files | ✅ Use file operations |
| ❌ No access to external systems | ✅ Integrate with databases, services |

**How it works:**
1. You give the LLM a list of available tools
2. The LLM decides when to use a tool
3. The LLM generates a tool call with parameters
4. The SDK executes the tool automatically
5. The result goes back to the LLM
6. The LLM incorporates it into the response

## 2️⃣ Built-in Tool: math_calculator

The `math_calculator` tool evaluates mathematical expressions safely.

In [ ]:
from local_llm_sdk import LocalLLMClient
from local_llm_sdk.tools import get_builtin_tools

# Create client and get built-in tools
client = LocalLLMClient(
    base_url="http://169.254.83.107:1234/v1",
    model="your-model-name"
)

tools = get_builtin_tools()

# Register tools with the client
client.register_tools(tools)

print("✅ Registered tools:")
for tool_name in client.tools.list_tools():
    print(f"  - {tool_name}")

Now let's use the calculator!

In [ ]:
# Ask a math question
response = client.chat("What is 127 multiplied by 893?")
print(response)

**🎉 Behind the scenes:**
1. The LLM recognized this needs calculation
2. It called `math_calculator(expression="127 * 893")`
3. The SDK executed the tool: `127 * 893 = 113411`
4. The LLM received the result
5. The LLM formatted a natural language response

All automatic!

Let's try more complex calculations:

In [ ]:
# Complex expression
response = client.chat("Calculate: (15 + 25) * 3 / 2")
print("Complex:", response)

# With context
response = client.chat(
    "If I have 12 boxes with 24 items each, and I sell them at $3.50 per item, "
    "how much revenue do I make?"
)
print("\nWith context:", response)

# Multiple calculations
response = client.chat(
    "What's the area of a rectangle with width 15.5 and height 23.7? "
    "And what's the perimeter?"
)
print("\nMultiple calculations:", response)

## 3️⃣ Built-in Tool: text_transformer

The `text_transformer` tool can uppercase, lowercase, reverse, or count words in text.

In [ ]:
# Uppercase transformation
response = client.chat("Convert 'hello world' to uppercase")
print("Uppercase:", response)

# Lowercase transformation
response = client.chat("Make 'PYTHON IS AWESOME' all lowercase")
print("\nLowercase:", response)

# Reverse text
response = client.chat("Reverse the text: 'artificial intelligence'")
print("\nReverse:", response)

# Count words
response = client.chat(
    "How many words are in this sentence: 'The quick brown fox jumps over the lazy dog'"
)
print("\nWord count:", response)

## 4️⃣ Built-in Tool: char_counter

The `char_counter` tool counts characters in text (with option to include/exclude spaces).

In [ ]:
# Count with spaces
response = client.chat("How many characters are in 'Hello, World!' including spaces?")
print("With spaces:", response)

# Count without spaces
response = client.chat("How many characters are in 'Hello, World!' excluding spaces?")
print("\nWithout spaces:", response)

# Compare lengths
response = client.chat(
    "Which is longer: 'supercalifragilisticexpialidocious' or 'antidisestablishmentarianism'? "
    "Tell me the character count of each."
)
print("\nComparison:", response)

## 5️⃣ Inspecting Tool Calls

To see exactly what tools were called, use `return_full_response=True`.

In [ ]:
# Get full response to see tool calls
response = client.chat(
    "What is 456 * 789 and also uppercase the word 'python'",
    return_full_response=True
)

print("📊 Response Analysis:\n")
print(f"Model: {response.model}")
print(f"Finish reason: {response.choices[0].finish_reason}")
print(f"\nFinal message: {response.choices[0].message.content}")

# Check if tools were called
message = response.choices[0].message
if message.tool_calls:
    print(f"\n🔧 Tool Calls Made: {len(message.tool_calls)}")
    for i, tool_call in enumerate(message.tool_calls, 1):
        print(f"\nTool Call {i}:")
        print(f"  Function: {tool_call.function.name}")
        print(f"  Arguments: {tool_call.function.arguments}")
        print(f"  ID: {tool_call.id}")
else:
    print("\n(No tools were called)")

**💡 Understanding the output:**

- `tool_calls`: List of all tools the LLM requested
- `function.name`: Which tool was called
- `function.arguments`: Parameters passed to the tool (JSON string)
- `id`: Unique identifier for this tool call

The SDK automatically executes these and sends results back to the LLM.

## 6️⃣ Combining Multiple Tools

The LLM can use multiple tools in a single conversation to solve complex tasks.

In [ ]:
# Complex request requiring multiple tools
response = client.chat(
    "I have a text: 'The Quick BROWN fox'. "
    "Make it all lowercase, count the characters (no spaces), "
    "and if the character count is even, multiply it by 5, otherwise multiply by 3."
)

print("Result:", response)

**🧠 The LLM orchestrated:**
1. `text_transformer` to lowercase
2. `char_counter` to count (without spaces)
3. `math_calculator` to multiply based on even/odd

This shows the power of tools - the LLM becomes a coordinator!

## 7️⃣ Tools with Conversation History

Tools work seamlessly with conversation history.

In [ ]:
# Start a conversation with tools
history = []

# Turn 1: Ask about a calculation
response1, history = client.chat_with_history(
    "What is 25 * 16?",
    history
)
print("Turn 1:")
print(f"You: What is 25 * 16?")
print(f"LLM: {response1}")

# Turn 2: Reference previous result
response2, history = client.chat_with_history(
    "Now add 100 to that result.",
    history
)
print("\nTurn 2:")
print(f"You: Now add 100 to that result.")
print(f"LLM: {response2}")

# Turn 3: More operations
response3, history = client.chat_with_history(
    "Convert that number to a string and count its characters.",
    history
)
print("\nTurn 3:")
print(f"You: Convert that number to a string and count its characters.")
print(f"LLM: {response3}")

**💡 Notice:**
- The LLM remembers previous calculations
- "that result" correctly refers to the earlier answer
- Tools are used across multiple conversation turns
- Context + Tools = Powerful interactions!

## 🏋️ Exercise: Multi-Tool Text Analyzer

**Challenge:** Create a conversation that:
1. Takes a sample text: "The LOCAL LLM SDK makes AI Development EASIER!"
2. Converts it to lowercase
3. Counts characters with and without spaces
4. Counts words
5. Calculates the average character-per-word ratio

Requirements:
- Use at least 3 different built-in tools
- Show each step of the analysis
- Display the final statistics

Try it yourself first!

In [ ]:
# Your code here:



<details>
<summary>Click to see solution</summary>

```python
# Solution: Multi-tool text analyzer

sample_text = "The LOCAL LLM SDK makes AI Development EASIER!"

print("📝 Text Analysis Tool Demo\n")
print(f"Original text: {sample_text}")
print("="*70 + "\n")

# Analysis request
response = client.chat(
    f"Analyze this text: '{sample_text}'. "
    f"First, convert it to lowercase. "
    f"Then tell me: "
    f"(1) how many characters including spaces, "
    f"(2) how many characters excluding spaces, "
    f"(3) how many words, and "
    f"(4) calculate the average characters per word (chars without spaces / word count)."
)

print("🔍 Analysis Results:")
print(response)

# Alternative: Step by step with conversation history
print("\n" + "="*70)
print("\n📊 Step-by-Step Analysis:\n")

history = []

# Step 1: Lowercase
response1, history = client.chat_with_history(
    f"Convert to lowercase: '{sample_text}'",
    history
)
print(f"Step 1 - Lowercase: {response1}")

# Step 2: Character counts
response2, history = client.chat_with_history(
    "Count characters in that lowercased text, both with and without spaces.",
    history
)
print(f"\nStep 2 - Char counts: {response2}")

# Step 3: Word count
response3, history = client.chat_with_history(
    "How many words are in it?",
    history
)
print(f"\nStep 3 - Word count: {response3}")

# Step 4: Average
response4, history = client.chat_with_history(
    "Calculate the average characters per word (using chars without spaces).",
    history
)
print(f"\nStep 4 - Average: {response4}")
```
</details>

In [ ]:
# Solution cell (run this to see the answer)
sample_text = "The LOCAL LLM SDK makes AI Development EASIER!"

print("📝 Text Analysis Tool Demo\n")
print(f"Original text: {sample_text}")
print("="*70 + "\n")

# One-shot analysis
response = client.chat(
    f"Analyze this text: '{sample_text}'. "
    f"First, convert it to lowercase. "
    f"Then tell me: "
    f"(1) how many characters including spaces, "
    f"(2) how many characters excluding spaces, "
    f"(3) how many words, and "
    f"(4) calculate the average characters per word (chars without spaces / word count)."
)

print("🔍 Analysis Results:")
print(response)

# Step-by-step version
print("\n" + "="*70)
print("\n📊 Step-by-Step Analysis:\n")

history = []

response1, history = client.chat_with_history(
    f"Convert to lowercase: '{sample_text}'",
    history
)
print(f"Step 1 - Lowercase: {response1}")

response2, history = client.chat_with_history(
    "Count characters in that lowercased text, both with and without spaces.",
    history
)
print(f"\nStep 2 - Char counts: {response2}")

response3, history = client.chat_with_history(
    "How many words are in it?",
    history
)
print(f"\nStep 3 - Word count: {response3}")

response4, history = client.chat_with_history(
    "Calculate the average characters per word (using chars without spaces).",
    history
)
print(f"\nStep 4 - Average: {response4}")

## ⚠️ Common Pitfalls

### 1. Forgetting to Register Tools
```python
# ❌ Bad: Tools not registered
client = LocalLLMClient(base_url="...", model="...")
response = client.chat("Calculate 5 * 5")
# LLM tries to do math itself (often wrong)

# ✅ Good: Register tools first
client = LocalLLMClient(base_url="...", model="...")
client.register_tools(get_builtin_tools())
response = client.chat("Calculate 5 * 5")
```

### 2. Using Tools for Simple Tasks
```python
# ❌ Overkill: Tool for "2+2"
response = client.chat("What is 2 + 2?")
# LLM might use calculator for trivial math

# ✅ Better: Let LLM handle simple cases
# For very simple operations, the LLM is fine
# Use tools for complex, precise, or multi-step calculations
```

### 3. Model Doesn't Support Function Calling
```python
# ⚠️ Some models don't support function calling well
# Check your model's capabilities:
# - Qwen, Hermes, Functionary: Good function calling
# - Older or smaller models: May not support it

# Test with a simple tool call to verify
response = client.chat("Calculate 123 * 456")
# If answer is wrong, model may not support tools
```

### 4. Not Handling Tool Errors
```python
# ❌ Bad: Unsafe expression
response = client.chat("Calculate: import os; os.system('rm -rf /')")
# math_calculator blocks this, but be aware

# ✅ Good: Tools have built-in safety
# math_calculator only allows math operations
# Always validate tool parameters in custom tools
```

## 🎓 What You Learned

✅ **Tool Concept**: Functions that extend LLM capabilities beyond text generation

✅ **Built-in Tools**: `math_calculator`, `text_transformer`, `char_counter`

✅ **Automatic Execution**: SDK handles tool calls transparently

✅ **Tool Inspection**: Use `return_full_response=True` to see tool_calls

✅ **Multi-Tool Tasks**: LLM can orchestrate multiple tools for complex operations

✅ **Tools + History**: Combine tools with conversation context for powerful workflows

✅ **Registration**: Must register tools with `client.register_tools()` before use

## 🚀 Next Steps

You've mastered built-in tools! Now let's create your own custom tools.

➡️ Continue to [05-custom-tools.ipynb](./05-custom-tools.ipynb) to learn how to:
- Create custom tools with the `@tool` decorator
- Define parameters with type hints
- Handle errors in tool functions
- Register and use your own tools
- Build a complete unit converter tool
- Follow best practices for tool design